# Visualisation
Various examples of model / performance evaluation

In [ ]:
from joblib import load
model = load('ADABoost_DS-0,1.joblib')

In [ ]:
import xarray as xr
from main.train.prepare_samples import prep_samples
ds = xr.open_zarr('./data/_ZARR_READY/canada-post')
X, y = prep_samples(
    ds, 
    compute_chunks = True
)

In [ ]:
import os

os.environ['SCIPY_ARRAY_API'] = '1'

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

ds = RandomUnderSampler(sampling_strategy = 0.1)
X_down, y_down = ds.fit_resample(X, y)

In [ ]:
y_pred = model.predict(X_down)

In [ ]:
import dask.array as da

full_arr = da.concatenate([X_down, y_down.reshape(-1, 1), y_pred.reshape(-1, 1)], axis=1)

In [ ]:
import dask.dataframe as df

ORIGINAL_FEATURES = ['d2m', 'lai_hv', 'lai_lv', 'mu_t2m_180', 'mu_t2m_30', 'mu_t2m_90', 'mu_tp_90', 'mu_tp_30', 'sp', 'mu_tp_180', 't2m', 'tp', 'ws10']
full_data = df.from_array(
    full_arr, columns = ORIGINAL_FEATURES + ['fire', 'predicted']
)

In [ ]:
import seaborn as sns

sns.boxenplot(
    full_data, x = 'fire', y = ''
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

x_var = 't2m'
y_var = 'mu_t2m_30'

plot_df = full_data[[x_var, y_var, 'fire']].compute()

sns.scatterplot(
    data = plot_df, x = x_var, y = y_var, hue = 'fire'
)
